In [1]:
import word2vec_wrapped
import doc2vec_wrapped
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import cross_val_score

c:\users\misko\miniconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-05-27 11:44:26,345 : INFO : loading projection weights from data/GoogleNews-vectors-negative300.bin
2018-05-27 11:44:29,409 : INFO : loaded (200000, 300) matrix from data/GoogleNews-vectors-negative300.bin


In [2]:
inception_df = pd.read_json("data/inception_output.txt", lines=True)
inception_df.set_index('image',inplace=True)
inception_df[:5]

,vector
image,
000000173830.jpg,"[0.27057743072509705, 0.11477771401405301, 0.0..."
000000570664.jpg,"[0.252127289772033, 0.434070438146591, 0.22712..."
000000304396.jpg,"[0.24179434776306102, 0.08653920888900701, 0.1..."
000000441491.jpg,"[0.11056154966354301, 0.08768206834793, 0.0857..."
000000289059.jpg,"[0.23210205137729603, 0.1578049659729, 0.27599..."


In [3]:
def concatenate(vector1, vector2, vector3 = 0):
    a = np.array(vector1)
    b = np.array(vector2)
    if (vector3 == 0):
        return np.concatenate((a, b))
    c = np.array(vector3)
    return np.concatenate((np.concatenate((a, b)), c))

In [4]:
def cross_val_accuracy(classifier, df):
    # Returns the mean accuracy on the given test data and labels, in 5 cross validation splits
    scores = cross_val_score(classifier, 
                             pd.DataFrame(df["concat_vectors"].tolist()), 
                             df["category"].values, 
                             cv=5)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# doc2vec_inception_class

In [5]:
def doc2vec_inception_class():
    df = pd.read_json("data/COCO/coco-easier.txt", 
                  lines=True)
    df = df.sample(frac=1).reset_index(drop=True)
    doc2vec_vectors_series = doc2vec_wrapped.vectorize_content()
    df["concat_vectors"] = df.apply(lambda row: concatenate(doc2vec_vectors_series[row['file_name']],
                                                           inception_df['vector'][row['file_name']]), axis=1)
    X = df["concat_vectors"]
    X

    from sklearn.neural_network import MLPClassifier
    X = df["concat_vectors"].tolist()
    y = df["category"].tolist()
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                        hidden_layer_sizes=(2500, 2500), random_state=1)
    clf.fit(X, y)                         
    MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
           beta_1=0.9, beta_2=0.999, early_stopping=False,
           epsilon=1e-08, hidden_layer_sizes=(5, 2), learning_rate='constant',
           learning_rate_init=0.001, max_iter=200, momentum=0.9,
           nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
           solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
           warm_start=False)
    print("doc2vec_inception")
    cross_val_accuracy(clf, df)

# word2vec_inception_class

In [6]:
def word2vec_inception_class():
    df = pd.read_json("data/COCO/coco-easier.txt", 
                  lines=True)
    df = df.sample(frac=1).reset_index(drop=True)
    word2vec_vectors_series = word2vec_wrapped.vectorize_content()
    df["concat_vectors"] = df.apply(lambda row: concatenate(word2vec_vectors_series[row['file_name']],
                                                           inception_df['vector'][row['file_name']]), axis=1)
    X = df["concat_vectors"]
    X

    from sklearn.neural_network import MLPClassifier
    X = df["concat_vectors"].tolist()
    y = df["category"].tolist()
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                        hidden_layer_sizes=(2500, 2500), random_state=1)
    clf.fit(X, y)                         
    MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
           beta_1=0.9, beta_2=0.999, early_stopping=False,
           epsilon=1e-08, hidden_layer_sizes=(5, 2), learning_rate='constant',
           learning_rate_init=0.001, max_iter=200, momentum=0.9,
           nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
           solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
           warm_start=False)
    print("word2vec_inception")
    cross_val_accuracy(clf, df)

# word2vec_doc2vec_class

In [7]:
def word2vec_doc2vec_class():
    df = pd.read_json("data/COCO/coco-easier.txt", 
                  lines=True)
    df = df.sample(frac=1).reset_index(drop=True)
    word2vec_vectors_series = word2vec_wrapped.vectorize_content()
    doc2vec_vectors_series = doc2vec_wrapped.vectorize_content()
    df["concat_vectors"] = df.apply(lambda row: concatenate(word2vec_vectors_series[row['file_name']],
                                                           doc2vec_vectors_series[row['file_name']]), axis=1)
    X = df["concat_vectors"]
    X

    from sklearn.neural_network import MLPClassifier
    X = df["concat_vectors"].tolist()
    y = df["category"].tolist()
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                        hidden_layer_sizes=(2500, 2500), random_state=1)
    clf.fit(X, y)                         
    MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
           beta_1=0.9, beta_2=0.999, early_stopping=False,
           epsilon=1e-08, hidden_layer_sizes=(5, 2), learning_rate='constant',
           learning_rate_init=0.001, max_iter=200, momentum=0.9,
           nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
           solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
           warm_start=False)
    print("word2vec_doc2vec")
    cross_val_accuracy(clf, df)

# word2vec_doc2vec_inception_class

In [8]:
def word2vec_doc2vec_inception_class():
    df = pd.read_json("data/COCO/coco-easier.txt", 
                  lines=True)
    df = df.sample(frac=1).reset_index(drop=True)
    word2vec_vectors_series = word2vec_wrapped.vectorize_content()
    doc2vec_vectors_series = doc2vec_wrapped.vectorize_content()

    df["concat_vectors"] = df.apply(lambda row: concatenate(word2vec_vectors_series[row['file_name']],
                                                            doc2vec_vectors_series[row['file_name']],
                                                            inception_df['vector'][row['file_name']]), axis=1)
  
    X = df["concat_vectors"]
    X

    from sklearn.neural_network import MLPClassifier
    X = df["concat_vectors"].tolist()
    y = df["category"].tolist()
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                        hidden_layer_sizes=(2500, 2500), random_state=1)
    clf.fit(X, y)                         
    MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
           beta_1=0.9, beta_2=0.999, early_stopping=False,
           epsilon=1e-08, hidden_layer_sizes=(5, 2), learning_rate='constant',
           learning_rate_init=0.001, max_iter=200, momentum=0.9,
           nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
           solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
           warm_start=False)
    print("word2vec_doc2vec_inception")
    cross_val_accuracy(clf, df)

In [9]:
doc2vec_inception_class()
word2vec_inception_class()
word2vec_doc2vec_class()
word2vec_doc2vec_inception_class()



2018-05-27 11:44:31,324 : INFO : collecting all words and their counts
2018-05-27 11:44:31,325 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-05-27 11:44:31,346 : INFO : collected 3702 word types and 10 unique tags from a corpus of 1503 examples and 65463 words
2018-05-27 11:44:31,347 : INFO : Loading a fresh vocabulary
2018-05-27 11:44:31,351 : INFO : min_count=5 retains 1058 unique words (28% of original 3702, drops 2644)
2018-05-27 11:44:31,352 : INFO : min_count=5 leaves 61217 word corpus (93% of original 65463, drops 4246)
2018-05-27 11:44:31,358 : INFO : deleting the raw counts dictionary of 3702 items
2018-05-27 11:44:31,359 : INFO : sample=0.001 downsamples 71 most-common words
2018-05-27 11:44:31,360 : INFO : downsampling leaves estimated 39737 word corpus (64.9% of prior 61217)
2018-05-27 11:44:31,366 : INFO : estimated required memory for 1058 words and 400 dimensions: 3932600 bytes
2018-05-27 11:44:31,367 : INFO : resetting layer weight

doc2vec_inception
Accuracy: 0.93 (+/- 0.01)
word2vec_inception


2018-05-27 12:16:34,764 : INFO : collecting all words and their counts
2018-05-27 12:16:34,769 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-05-27 12:16:34,800 : INFO : collected 3702 word types and 10 unique tags from a corpus of 1503 examples and 65463 words
2018-05-27 12:16:34,801 : INFO : Loading a fresh vocabulary
2018-05-27 12:16:34,808 : INFO : min_count=5 retains 1058 unique words (28% of original 3702, drops 2644)
2018-05-27 12:16:34,810 : INFO : min_count=5 leaves 61217 word corpus (93% of original 65463, drops 4246)
2018-05-27 12:16:34,817 : INFO : deleting the raw counts dictionary of 3702 items
2018-05-27 12:16:34,818 : INFO : sample=0.001 downsamples 71 most-common words
2018-05-27 12:16:34,819 : INFO : downsampling leaves estimated 39737 word corpus (64.9% of prior 61217)
2018-05-27 12:16:34,826 : INFO : estimated required memory for 1058 words and 400 dimensions: 3932600 bytes
2018-05-27 12:16:34,828 : INFO : resetting layer weight

Accuracy: 0.90 (+/- 0.03)


2018-05-27 12:16:35,091 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-27 12:16:35,092 : INFO : EPOCH - 1 : training on 65463 raw words (41232 effective words) took 0.2s, 189660 effective words/s
2018-05-27 12:16:35,293 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-27 12:16:35,295 : INFO : EPOCH - 2 : training on 65463 raw words (41111 effective words) took 0.2s, 205945 effective words/s
2018-05-27 12:16:35,500 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-27 12:16:35,501 : INFO : EPOCH - 3 : training on 65463 raw words (41258 effective words) took 0.2s, 203100 effective words/s
2018-05-27 12:16:35,694 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-27 12:16:35,695 : INFO : EPOCH - 4 : training on 65463 raw words (41138 effective words) took 0.2s, 216100 effective words/s
2018-05-27 12:16:35,890 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-27 12

word2vec_doc2vec


2018-05-27 12:33:10,196 : INFO : collecting all words and their counts
2018-05-27 12:33:10,197 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-05-27 12:33:10,218 : INFO : collected 3702 word types and 10 unique tags from a corpus of 1503 examples and 65463 words
2018-05-27 12:33:10,219 : INFO : Loading a fresh vocabulary
2018-05-27 12:33:10,225 : INFO : min_count=5 retains 1058 unique words (28% of original 3702, drops 2644)
2018-05-27 12:33:10,226 : INFO : min_count=5 leaves 61217 word corpus (93% of original 65463, drops 4246)
2018-05-27 12:33:10,232 : INFO : deleting the raw counts dictionary of 3702 items
2018-05-27 12:33:10,233 : INFO : sample=0.001 downsamples 71 most-common words
2018-05-27 12:33:10,234 : INFO : downsampling leaves estimated 39737 word corpus (64.9% of prior 61217)
2018-05-27 12:33:10,238 : INFO : estimated required memory for 1058 words and 400 dimensions: 3932600 bytes
2018-05-27 12:33:10,240 : INFO : resetting layer weight

Accuracy: 0.96 (+/- 0.02)


2018-05-27 12:33:10,488 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-27 12:33:10,490 : INFO : EPOCH - 1 : training on 65463 raw words (41232 effective words) took 0.2s, 191649 effective words/s
2018-05-27 12:33:10,692 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-27 12:33:10,694 : INFO : EPOCH - 2 : training on 65463 raw words (41111 effective words) took 0.2s, 205451 effective words/s
2018-05-27 12:33:10,899 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-27 12:33:10,900 : INFO : EPOCH - 3 : training on 65463 raw words (41258 effective words) took 0.2s, 204377 effective words/s
2018-05-27 12:33:11,093 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-27 12:33:11,094 : INFO : EPOCH - 4 : training on 65463 raw words (41138 effective words) took 0.2s, 215600 effective words/s
2018-05-27 12:33:11,294 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-27 12

word2vec_doc2vec_inception
Accuracy: 0.93 (+/- 0.03)
